In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
#import matplotlib.pyplot as plt
import numpy as np

import os
import shutil
#import pandas
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error
import math


import tensorflow as tf

from tensorflow.keras import layers
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)



In [3]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()


In [50]:
import tensorflow_text as text
texto="En en En EN vista de la situación que atraviesa el país por cuenta del virus Covid-19 y luego de la solicitud directa del alcalde Daniel Quintero con la intención de proteger la vida la los ciudadanos, los miembros del Consejo Municipal de Gestión del Riesgo decidieron declarar la Calamidad Pública y Emergencia Sanitaria." #53 palabras

t=tf.keras.preprocessing.text.Tokenizer(char_level=False)
t.fit_on_texts(texto)
m=t.texts_to_matrix(texto)
s=t.texts_to_sequences(texto)
#te=t.texts_to_sequences_generator(texto)
print(t.sequences_to_texts([[1], [6]]))
print(s)

palabras=text.Tokenizer(texto)
print(palabras)



ModuleNotFoundError: No module named 'tensorflow_text'

In [4]:
def insertarPalabra(palabra, palabras, posiciones, minuscula):
    if len(palabra)==0:
        return
    if minuscula:
        palabra=palabra.lower()
    if(palabra not in palabras):
        palabras.append(palabra)
    indice=palabras.index(palabra)
    posiciones.append(indice)
def vocabulario(fichero, caracterSeparacion=' ', caracteresPuntuacion=['!','"','#','$','%','&','(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\','\'',']','^','_','`','{','|','}','~','\t','\n','\r','¿','¡'],caracteresDescartados=[], minuscula=True, palabras=[]):
    posiciones=[]

    with open(fichero,mode="r",encoding="utf8") as fichero:
        palabra=""
        esPuntuacion=False
        for linea in fichero:
    
            for c in linea:
                if(c==caracterSeparacion):
                    if(esPuntuacion):
                        palabra+=c
                    insertarPalabra(palabra, palabras, posiciones, minuscula)
                    palabra=""
                    esPuntuacion=False
                else:
                    if(c not in caracteresPuntuacion):
                        if(esPuntuacion):
                            insertarPalabra(palabra, palabras, posiciones,minuscula)
                            esPuntuacion=False
                            palabra=c
                        else:
                            palabra+=c
                    else:
                        insertarPalabra(palabra, palabras, posiciones,minuscula)
                        palabra=c
                        esPuntuacion=True
    return palabras,posiciones

In [13]:
def dividir(posiciones,longitud_bloque):
    conjunto=[]
    elemento=0
    bloque=[]
    for i,posicion in enumerate(posiciones):
        if elemento==10:
            if(len(bloque)>0):
                conjunto.append(bloque)
                bloque=[]
                elemento=0
        bloque.append(posicion)
        elemento+=1
    if(len(bloque)>0):
        longitud=len(bloque)
        for i in range(longitud_bloque-longitud):
            bloque.append(0)
        conjunto.append(bloque)
    return conjunto

In [15]:
palabras,posiciones=vocabulario("textos\libro.txt", palabras=["\n"])

numeroEntrenamiento=math.floor(len(posiciones)*0.8)
entrenamiento=posiciones[:numeroEntrenamiento]
test=posiciones[numeroEntrenamiento:]

longitud_bloque=10
entrenamiento=dividir(entrenamiento,longitud_bloque)
test=dividir(test,longitud_bloque)


print("Vocabulario=",len(palabras))
print("Entrenamiento=",entrenamiento.shape)
print("test=",len(test))


Vocabulario= 7460


AttributeError: 'list' object has no attribute 'shape'

In [105]:
#embedding_layer = layers.Embedding(len(posiciones), 2)
#result = embedding_layer(tf.constant(posiciones))
#result.numpy()


array([[-0.00512495,  0.04321748],
       [ 0.03834225, -0.01898133],
       [ 0.00431837, -0.03773835],
       ...,
       [-0.03060499,  0.01168156],
       [ 0.0071156 , -0.02809701],
       [-0.0082266 ,  0.03553892]], dtype=float32)

In [118]:
def crearModelo(longitud_vocabulario,longitud_embedding=16 ):
    model = tf.keras.Sequential([
    layers.Embedding(longitud_vocabulario, longitud_embedding),
    layers.GlobalAveragePooling1D(),
    layers.Dense(longitud_embedding, activation='relu'),
    layers.Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model


In [122]:
modelo=crearModelo(len(palabras))

history = modelo.fit(
    entrenamiento,
    epochs=10,
    validation_data=test, validation_steps=20)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()